In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [11]:
# Set the paths to your rock, paper, and scissor folders
rock_folder = 'rock/'
paper_folder = 'paper/'
scissor_folder = 'scissors/'

In [12]:
# Parameters for resizing images
image_width = 128  # Desired width
image_height = 128  # Desired height

In [13]:
# Function to extract HOG features from an image
def extract_hog_features(image):
    hog = cv2.HOGDescriptor()
    hog_features = hog.compute(image)
    hog_features = hog_features.reshape(-1)  # Flatten the feature vector
    return hog_features

In [14]:
# Load and process the images
X = []
y = []

In [15]:
# Load rock images
print('started:', rock_folder)
for filename in os.listdir(rock_folder):
    if filename.endswith('.png'):
        image = cv2.imread(os.path.join(rock_folder, filename))
        image = cv2.resize(image, (image_width, image_height))  # Resize the image
        hog_features = extract_hog_features(image)
        X.append(hog_features)
        y.append(0)  # Rock class label
print('ended:', rock_folder)

started: rock/
ended: rock/


In [16]:
# Load paper images
print('started:', paper_folder)
for filename in os.listdir(paper_folder):
    if filename.endswith('.png'):
        image = cv2.imread(os.path.join(paper_folder, filename))
        image = cv2.resize(image, (image_width, image_height))  # Resize the image
        hog_features = extract_hog_features(image)
        X.append(hog_features)
        y.append(1)  # Paper class label
print('ended:', paper_folder)

started: paper/
ended: paper/


In [17]:
# Load scissor images
print('started:', scissor_folder)
for filename in os.listdir(scissor_folder):
    if filename.endswith('.png'):
        image = cv2.imread(os.path.join(scissor_folder, filename))
        image = cv2.resize(image, (image_width, image_height))  # Resize the image
        hog_features = extract_hog_features(image)
        X.append(hog_features)
        y.append(2)  # Scissor class label
print('ended:', scissor_folder)

started: scissors/
ended: scissors/


In [18]:
# Convert the feature matrix and labels to numpy arrays
X = np.array(X)
y = np.array(y)

In [19]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [49]:
# Create the decision tree classifier
model = DecisionTreeClassifier()

# Train the model on the training set
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [51]:
# Make predictions on the test set
y_pred = model.predict(X_test)

In [58]:
# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8036529680365296


In [24]:
import joblib

In [69]:
joblib.dump(model, 'rock_paper_scissor_decision_tree.joblib')

['rock_paper_scissor_decision_tree2.joblib']

In [21]:
from sklearn.ensemble import RandomForestClassifier  # For classification

In [53]:
model2 = RandomForestClassifier()

In [54]:
model2.fit(X_train, y_train)

RandomForestClassifier()

In [55]:
y_pred = model2.predict(X_test)

In [58]:
# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.952054794520548


In [35]:
# save the model
joblib.dump(model2, 'rock_paper_scissor_random_forest.joblib')

['rock_paper_scissor_random_forest.joblib']

In [32]:
# Train KNeighborsClassifier model
from sklearn.neighbors import KNeighborsClassifier
import math

In [33]:
# this is the best solution to provide k value for the KNearestNeighbors algorithm (Should be odd number)
math.sqrt(len(y_train))

41.83300132670378

In [35]:
# this will use k value as 41 taken from previous method, p is 3 as we have 3 class, metric euclidean to calculate distance between the neighbors
KNearestNeighbors = KNeighborsClassifier(n_neighbors=41, p=3, metric='euclidean') 
KNearestNeighbors.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=41, p=3)

In [36]:
y_pred_of_knn = KNearestNeighbors.predict(X_test)

In [47]:
# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred_of_knn)
print("Accuracy:", accuracy)

Accuracy: 0.9634703196347032


In [50]:
# save the model
joblib.dump(KNearestNeighbors, 'knn.joblib')

['knn.joblib']

In [20]:
# using gaussian naive bayes
from sklearn.naive_bayes import GaussianNB
gaussianNB_model = GaussianNB()
gaussianNB_model.fit(X_train, y_train)

GaussianNB()

In [21]:
y_pred_of_gnb = gaussianNB_model.predict(X_test)

In [22]:
# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred_of_gnb)
print("Accuracy:", accuracy)

Accuracy: 0.9041095890410958


In [25]:
# save the model
joblib.dump(gaussianNB_model, 'gnb.joblib')

['gnb.joblib']